In [9]:
#!kaggle competitions download -c tweet-sentiment-extraction

100%|██████████████████████████████████████| 1.39M/1.39M [00:01<00:00, 1.20MB/s]
100%|██████████████████████████████████████| 1.39M/1.39M [00:01<00:00, 1.20MB/s]


In [3]:
import pandas as pd
import numpy as np
import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *


In [4]:
train = pd.read_csv("/Users/reejungkim/Documents/Git/Sentimental analysis/tweet-sentiment-extraction/train.csv")
test = pd.read_csv("/Users/reejungkim/Documents/Git/Sentimental analysis/tweet-sentiment-extraction/test.csv")

In [5]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [6]:
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


# Data exploration

## checking for null values

In [10]:
train.isnull().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [13]:
train.loc[pd.isnull(train.text)]

,textID,text,selected_text,sentiment
314,fdb77c3752,NaN,NaN,neutral


## drop null

In [15]:
train.dropna(inplace=True)

## count of unique values

In [16]:
train.sentiment.unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [17]:
train.shape, test.shape

((27480, 4), (3534, 3))

In [45]:
train.pivot_table(index='sentiment', values='textID', 
                  aggfunc='count', margins=True)


,textID
sentiment,
negative,7781
neutral,11117
positive,8582
All,27480


In [46]:
train.pivot_table(index='sentiment', values='textID', 
                 aggfunc = lambda x: x.count()/train['sentiment'].count())


,textID
sentiment,
negative,0.283151
neutral,0.404549
positive,0.312300


In [18]:

n_instances = 100